In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import geopandas as gpd
import contextily as ctx
from shapely.geometry import Point
import os
import sys

# 设置配置
class Config:
    # 使用绝对路径
    ROOT_DIR = "D:/Project/ML/Singapore cloud project"
    RESULT_DIR = os.path.join(ROOT_DIR, "result")
    DATA_DIR = os.path.join(ROOT_DIR, "data/processed")
    MODEL_PATH = os.path.join(RESULT_DIR, "solar_model.pth")
    THRESHOLD_PERCENTILE = 80
    FIGURE_SIZE = (20, 10)

# 确保结果目录存在
os.makedirs(Config.RESULT_DIR, exist_ok=True)
    
# 设置matplotlib中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 自定义颜色映射
colors = ['#FF0000', '#FFFF00', '#00FF00']  # 红黄绿
cmap = LinearSegmentedColormap.from_list('custom_diverging', colors, N=100)

def load_model_and_data():
    """加载模型和数据"""
    try:
        # 检查文件是否存在
        if not os.path.exists(Config.MODEL_PATH):
            raise FileNotFoundError(f"模型文件不存在: {Config.MODEL_PATH}")
            
        # 加载模型
        model = SolarNet()
        model.load_state_dict(torch.load(Config.MODEL_PATH, weights_only=True))
        model.eval()
        
        # 检查数据文件
        cloud_path = os.path.join(Config.DATA_DIR, "avg_cloud_thickness.npy")
        slope_path = os.path.join(Config.DATA_DIR, "slope_resampled.npy")
        
        if not os.path.exists(cloud_path):
            raise FileNotFoundError(f"云层数据文件不存在: {cloud_path}")
        if not os.path.exists(slope_path):
            raise FileNotFoundError(f"坡度数据文件不存在: {slope_path}")
            
        # 加载数据
        cloud_data = np.load(cloud_path)
        slope_data = np.load(slope_path)
        
        return model, cloud_data, slope_data
        
    except Exception as e:
        print(f"加载模型或数据时出错: {str(e)}")
        raise

def generate_predictions(model, cloud_data, slope_data):
    """使用模型生成预测"""
    try:
        X = np.stack([cloud_data, slope_data], axis=0)
        X = torch.tensor(X, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            predictions = model(X).squeeze().numpy()
        return predictions
    except Exception as e:
        print(f"生成预测时出错: {str(e)}")
        raise

sys.path.append(Config.ROOT_DIR)
from solarnet import SolarNet

def analyze_predictions(predictions):
    """分析预测结果"""
    try:
        # 计算阈值
        threshold = np.percentile(predictions, Config.THRESHOLD_PERCENTILE)
        suitable_mask = predictions > threshold
        
        # 生成经纬度网格
        latitudes = np.linspace(1.2, 1.5, predictions.shape[0])
        longitudes = np.linspace(103.6, 104.1, predictions.shape[1])
        lat_grid, lon_grid = np.meshgrid(latitudes, longitudes, indexing='ij')
        
        # 创建适合位置的DataFrame
        suitable_locations = pd.DataFrame({
            'longitude': lon_grid[suitable_mask],
            'latitude': lat_grid[suitable_mask],
            'suitability_score': predictions[suitable_mask]
        })
        
        # 归一化得分
        suitable_locations['normalized_score'] = (
            (suitable_locations['suitability_score'] - predictions.min()) / 
            (predictions.max() - predictions.min())
        )
        
        return suitable_locations, threshold
    except Exception as e:
        print(f"分析预测结果时出错: {str(e)}")
        raise

def create_visualizations(predictions, suitable_locations):
    """创建可视化图表"""
    try:
        fig = plt.figure(figsize=Config.FIGURE_SIZE)
        
        # 1. 热图
        ax1 = plt.subplot(131)
        im = ax1.imshow(predictions, cmap=cmap)
        plt.colorbar(im, label='预测值')
        ax1.set_title('预测值分布热图')
        
        # 2. 散点图
        ax2 = plt.subplot(132)
        scatter = ax2.scatter(suitable_locations['longitude'], 
                            suitable_locations['latitude'],
                            c=suitable_locations['normalized_score'],
                            cmap=cmap,
                            s=50,
                            alpha=0.6)
        plt.colorbar(scatter, label='归一化适宜度得分')
        ax2.set_title('适合位置分布图')
        ax2.set_xlabel('经度')
        ax2.set_ylabel('纬度')
        
        # 3. 地图可视化
        ax3 = plt.subplot(133)
        visualize_on_map(suitable_locations, ax3)
        
        plt.suptitle('新加坡太阳能电站选址分析结果', fontsize=16, y=1.02)
        plt.tight_layout()
        return fig
    except Exception as e:
        print(f"创建可视化时出错: {str(e)}")
        raise

def visualize_on_map(suitable_locations, ax):
    """在地图上可视化最佳位置"""
    try:
        # 获取前10个最佳位置
        top_10_locations = suitable_locations.nlargest(10, 'suitability_score')
        geometry = [Point(xy) for xy in zip(top_10_locations['longitude'], 
                                          top_10_locations['latitude'])]
        gdf = gpd.GeoDataFrame(top_10_locations, geometry=geometry, crs='EPSG:4326')
        gdf = gdf.to_crs(epsg=3857)
        
        # 绘制点位
        gdf.plot(ax=ax, color='red', markersize=100, zorder=2)
        ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
        
        # 添加标注
        for idx, row in gdf.iterrows():
            ax.annotate(f'#{idx+1}\n得分: {row.suitability_score:.4f}',
                       xy=(row.geometry.x, row.geometry.y),
                       xytext=(10, 10),
                       textcoords='offset points',
                       color='red',
                       fontsize=8,
                       bbox=dict(facecolor='white', alpha=0.7))
        
        ax.set_title('前十个最适合位置')
        ax.set_axis_off()
    except Exception as e:
        print(f"地图可视化时出错: {str(e)}")
        raise

def save_results(suitable_locations, fig):
    """保存结果"""
    try:
        # 保存CSV文件
        csv_path = os.path.join(Config.RESULT_DIR, "suitable_locations.csv")
        suitable_locations.to_csv(csv_path, index=False)
        print(f"结果已保存至: {csv_path}")
        
        # 保存图像
        fig_path = os.path.join(Config.RESULT_DIR, "evaluation_results.png")
        fig.savefig(fig_path, dpi=300, bbox_inches='tight')
        print(f"可视化结果已保存至: {fig_path}")
        plt.close()
    except Exception as e:
        print(f"保存结果时出错: {str(e)}")
        raise

def main():
    """主函数"""
    try:
        print("开始执行评估和可视化...")
        print(f"当前工作目录: {os.getcwd()}")
        print(f"模型路径: {Config.MODEL_PATH}")
        print(f"数据目录: {Config.DATA_DIR}")
        
        # 1. 加载模型和数据
        model, cloud_data, slope_data = load_model_and_data()
        print("模型和数据加载成功")
        
        # 2. 生成预测
        predictions = generate_predictions(model, cloud_data, slope_data)
        print("预测生成成功")
        
        # 3. 分析预测结果
        suitable_locations, threshold = analyze_predictions(predictions)
        print("预测分析完成")
        
        # 4. 创建可视化
        fig = create_visualizations(predictions, suitable_locations)
        print("可视化创建完成")
        
        # 5. 保存结果
        save_results(suitable_locations, fig)
        print("结果保存完成")
        
        # 6. 输出统计信息
        print(f"\n使用阈值（{Config.THRESHOLD_PERCENTILE}百分位）：{threshold:.4f}")
        print("\n预测结果统计：")
        print(f"找到的适合位置数量：{len(suitable_locations)}")
        print(f"最小预测值：{predictions.min():.4f}")
        print(f"最大预测值：{predictions.max():.4f}")
        print(f"平均预测值：{predictions.mean():.4f}")
        
    except Exception as e:
        print(f"\n错误：{str(e)}")
        print("\n请检查以下几点：")
        print("1. 确保已安装必要的包")
        print("2. 确保模型文件存在")
        print("3. 确保数据文件路径正确")
        print("4. 检查网络连接（用于下载地图底图）")
        raise

if __name__ == "__main__":
    main()

开始执行评估和可视化...
当前工作目录: D:\Project\ML\Singapore cloud project
模型路径: D:/Project/ML/Singapore cloud project\result\solar_model.pth
数据目录: D:/Project/ML/Singapore cloud project\data/processed
模型和数据加载成功
预测生成成功
预测分析完成
可视化创建完成
结果已保存至: D:/Project/ML/Singapore cloud project\result\suitable_locations.csv
可视化结果已保存至: D:/Project/ML/Singapore cloud project\result\evaluation_results.png
结果保存完成

使用阈值（80百分位）：0.0183

预测结果统计：
找到的适合位置数量：307
最小预测值：0.0000
最大预测值：0.9457
平均预测值：0.0325
